In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
seed_everything(0)

In [5]:
use_contrastive = True
num_tasks = 4

data_cfg = {
    "dataset_name": "mnist",
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
}
dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)


In [8]:
seed_everything(7)
sender_dataset = get_dataset(**data_cfg)
# send_data = sender_dataset.trainset[0]

INFO:root:Class sequence: [9 5 7 8 4 3 6 1 0 2]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)


In [9]:
net_cfg = {
    'depth': 4,
    'layer_size': 64,
    'num_init_tasks': num_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': 4,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
}

In [11]:
NetCls = MLPSoftLLDynamic
LearnerCls = CompositionalDynamicER
sharing_cfg = DictConfig({
    "scorer": "cross_entropy",
    "num_queries": 4,
    "query_score_threshold": 0.0,
})
train_cfg = {
    # "num_epochs": 40,
    "num_epochs": 1,
}

agent = RecvDataAgent(0, 0, dataset,
                NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, 
                sharing_cfg)

INFO:root:Agent: node_id: 0, seed: 0


In [13]:
send_data = sender_dataset.trainset[0]
send_data

In [14]:
sender_sequence = sender_dataset.class_sequence
agent_sequence = dataset.class_sequence
label_mapping = {sender_sequence[i]: agent_sequence[i] for i in range(len(sender_sequence))}
label_mapping

{9: 5, 5: 0, 7: 3, 8: 4, 4: 2, 3: 6, 6: 7, 1: 1, 0: 8, 2: 9}

In [12]:
def global_label_utilization(agent, send_data, sender_dataset, dataset):
    # Create a label mapping based on the class sequences
    sender_sequence = sender_dataset.class_sequence
    agent_sequence = dataset.class_sequence

    label_mapping = {sender_sequence[i]: agent_sequence[i] for i in range(len(sender_sequence))}

    for t in range(num_tasks):
        # Extract data corresponding to the current task from send_data
        task_indices = (send_data[1] == t).nonzero(as_tuple=True)[0]
        X_task = send_data[0][task_indices]
        Y_task = send_data[1][task_indices]
        
        # Convert the sender's labels to agent's labels using global label mapping
        Y_mapped = torch.tensor([label_mapping[y.item()] for y in Y_task])
        
        # Fit the model
        agent.train_on_new_task(X_task, Y_mapped, task_id=t)

# Train the agent using global label approach
global_label_utilization(agent, send_data, sender_dataset, dataset)


AttributeError: 'bool' object has no attribute 'nonzero'